In [ ]:
# pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 57.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e13bbd8a92ef0a2480f757011f3f958f9535d918d09cca2a8a55d64046569c11
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession


spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_5")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
    
sc = spark.sparkContext

23/01/15 16:34:18 WARN Utils: Your hostname, ubuntu-d resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface enp7s0)
23/01/15 16:34:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/15 16:34:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Оконные функции

Оконная функция - функция, которая работает с выделенным набором строк (окном, партицией) и выполняет вычисление для этого набора строк в отдельном столбце. 

Партиции (окна из набора строк) - это набор строк, указанный для оконной функции по одному из столбцов или группе столбцов таблицы. Партиции для каждой оконной функции в запросе могут быть разделены по различным колонкам таблицы.

In [2]:
# Предположим, что есть таблицы:
from pyspark.sql import functions as F
from pyspark.sql.types import *

data =[
    ("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"),\
    ("Orange",2000,"USA"), ("Orange",2000,"USA"), ("Banana",400,"China"),\
    ("Carrots",1200,"China"), ("Beans",1500,"China"), ("Orange",4000,"China"),\
    ("Banana",2000,"Canada"), ("Carrots",2000,"Canada"), ("Beans",2000,"Mexico")
    ]
columns = ["Product","Amount", "Country"]

df = spark.createDataFrame(data=data, schema=columns)
df.printSchema()
df.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)



+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
|Banana |1000  |USA    |
|Carrots|1500  |USA    |
|Beans  |1600  |USA    |
|Orange |2000  |USA    |
|Orange |2000  |USA    |
|Banana |400   |China  |
|Carrots|1200  |China  |
|Beans  |1500  |China  |
|Orange |4000  |China  |
|Banana |2000  |Canada |
|Carrots|2000  |Canada |
|Beans  |2000  |Mexico |
+-------+------+-------+



In [3]:
# в sql оконная функция записывается следующим образом
df.createOrReplaceTempView('df')

spark.sql('''
select *, 
row_number() over( partition by Country order by Amount ) as rn from df
''').show()

+-------+------+-------+---+
|Product|Amount|Country| rn|
+-------+------+-------+---+
| Banana|  2000| Canada|  1|
|Carrots|  2000| Canada|  2|
| Banana|   400|  China|  1|
|Carrots|  1200|  China|  2|
|  Beans|  1500|  China|  3|
| Orange|  4000|  China|  4|
|  Beans|  2000| Mexico|  1|
| Banana|  1000|    USA|  1|
|Carrots|  1500|    USA|  2|
|  Beans|  1600|    USA|  3|
| Orange|  2000|    USA|  4|
| Orange|  2000|    USA|  5|
+-------+------+-------+---+



In [4]:
# в спарке следующим образом
from pyspark.sql import Window


windSpec = Window()\
    .partitionBy('Country')\
    .orderBy('Amount')
    # .rowsBetween(Window.unboundedPreceding, Window.currentRow - 1)

df.withColumn('rn', F.row_number().over(windSpec)).show()

+-------+------+-------+---+
|Product|Amount|Country| rn|
+-------+------+-------+---+
| Banana|  2000| Canada|  1|
|Carrots|  2000| Canada|  2|
| Banana|   400|  China|  1|
|Carrots|  1200|  China|  2|
|  Beans|  1500|  China|  3|
| Orange|  4000|  China|  4|
|  Beans|  2000| Mexico|  1|
| Banana|  1000|    USA|  1|
|Carrots|  1500|    USA|  2|
|  Beans|  1600|    USA|  3|
| Orange|  2000|    USA|  4|
| Orange|  2000|    USA|  5|
+-------+------+-------+---+



# Самостоятельная работа к уроку
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

In [ ]:
# !wget 'https://drive.google.com/uc?export=download&id=1xbtFBPz50OBoyYFVGd-B03pCTJdCax07' -O raw_sales.csv

--2022-09-23 17:45:26--  https://drive.google.com/uc?export=download&id=1xbtFBPz50OBoyYFVGd-B03pCTJdCax07
Resolving drive.google.com (drive.google.com)... 74.125.134.100, 74.125.134.101, 74.125.134.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.134.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/985fbke648pmgb8v0kmclemkqv6nsg1k/1663955100000/04099736791713398091/*/1xbtFBPz50OBoyYFVGd-B03pCTJdCax07?e=download&uuid=dda96373-c78e-47fb-8000-c4a2ddcc7543 [following]
--2022-09-23 17:45:27--  https://doc-0c-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/985fbke648pmgb8v0kmclemkqv6nsg1k/1663955100000/04099736791713398091/*/1xbtFBPz50OBoyYFVGd-B03pCTJdCax07?e=download&uuid=dda96373-c78e-47fb-8000-c4a2ddcc7543
Resolving doc-0c-5k-docs.googleusercontent.com (doc-0c-5k-docs.googleusercontent.com)... 74.125.196.132, 2607:

In [10]:
df = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)
df.printSchema()
df.show(truncate=False)

root
 |-- datesold: timestamp (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- propertyType: string (nullable = true)
 |-- bedrooms: integer (nullable = true)

+-------------------+--------+-------+------------+--------+
|datesold           |postcode|price  |propertyType|bedrooms|
+-------------------+--------+-------+------------+--------+
|2007-02-07 00:00:00|2607    |525000 |house       |4       |
|2007-02-27 00:00:00|2906    |290000 |house       |3       |
|2007-03-07 00:00:00|2905    |328000 |house       |3       |
|2007-03-09 00:00:00|2905    |380000 |house       |4       |
|2007-03-21 00:00:00|2906    |310000 |house       |3       |
|2007-04-04 00:00:00|2905    |465000 |house       |4       |
|2007-04-24 00:00:00|2607    |399000 |house       |3       |
|2007-04-30 00:00:00|2606    |1530000|house       |4       |
|2007-05-24 00:00:00|2902    |359000 |house       |3       |
|2007-05-25 00:00:00|2906    |320000 |house       |3   

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)
*  Стоимость последнего проданного дома до текущего


In [11]:
# 10 до текущего
windSpec_before = (Window().partitionBy('postcode')
                         .orderBy('datesold')
                         .rowsBetween(Window.currentRow - 10, Window.currentRow - 1)
                  )

# 10 после текущего
windSpec_after = (Window().partitionBy('postcode')
                          .orderBy('datesold')
                          .rowsBetween(Window.currentRow + 1, Window.currentRow + 10)
                 )

# последний до текущего
windSpec_last = (Window().partitionBy('postcode')
                         .orderBy('datesold')
                )

df = df.select('*', 
               F.avg('price').over(windSpec_before).alias('before'),
               F.avg('price').over(windSpec_after).alias('after'),
               F.lag('price', 1).over(windSpec_last).alias('last')
              )

In [12]:
df.select('datesold', 'postcode', 'price', 'before', 'after', 'last').show()

+-------------------+--------+-------+-----------------+--------+-------+
|           datesold|postcode|  price|           before|   after|   last|
+-------------------+--------+-------+-----------------+--------+-------+
|2007-07-08 00:00:00|    2600| 327000|             null|708350.0|   null|
|2007-08-16 00:00:00|    2600| 790000|         327000.0|698350.0| 327000|
|2007-12-05 00:00:00|    2600| 825000|         558500.0|679350.0| 790000|
|2008-01-21 00:00:00|    2600| 315000|647333.3333333334|742850.0| 825000|
|2008-04-24 00:00:00|    2600| 292500|         564250.0|786600.0| 315000|
|2008-05-30 00:00:00|    2600| 329000|         509900.0|839200.0| 292500|
|2008-06-19 00:00:00|    2600| 765000|         479750.0|868450.0| 329000|
|2008-07-29 00:00:00|    2600| 927000|         520500.0|805750.0| 765000|
|2008-09-02 00:00:00|    2600|1380000|         571312.5|715250.0| 927000|
|2008-09-08 00:00:00|    2600| 740000|661166.6666666666|756250.0|1380000|
|2008-09-17 00:00:00|    2600| 720000|

## Задание 2
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf.

In [15]:
# udf
@F.udf(returnType=IntegerType())
def row_distinct(row):
    return len(set(row))


df.select(row_distinct(F.array('*')).alias('distinkt_val')).show(5)

+------------+
|distinkt_val|
+------------+
|           7|
|           8|
|           8|
|           8|
|           8|
+------------+
only showing top 5 rows



In [17]:
# pandas_udf
import pandas as pd


@F.pandas_udf(IntegerType())
def pd_row_distinct(*row):
    tmp = pd.concat([*row], axis=1).apply(lambda x: x.unique().size, axis=1)
    return tmp


df.select(pd_row_distinct(*df.columns).alias('distinct_val')).show(5)

+------------+
|distinct_val|
+------------+
|           7|
|           7|
|           8|
|           8|
|           8|
+------------+
only showing top 5 rows



# Задание 3
SQL like case when или if elif else

Создайте колонку, в которой в которой будет отображаться "+", "-" или "=", если "Средняя стомость 10 проданных домов до текущего в том же районе" больше, меньше или равно "Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)", соотвественно.

Если одно из полей Null, запишите в эту колонку "Нет данных"

In [18]:
(df.select('price', 'before', 'after', 'last')
   .withColumn('if', 
               F.when(df.before > df.after, F.lit('+'))
                .when(df.before < df.after, F.lit('-'))
                .when(df.before == df.after, F.lit('='))
                .otherwise(F.lit('Нет данных')))
   .show()
)

+-------+-----------------+--------+-------+----------+
|  price|           before|   after|   last|        if|
+-------+-----------------+--------+-------+----------+
| 327000|             null|708350.0|   null|Нет данных|
| 790000|         327000.0|698350.0| 327000|         -|
| 825000|         558500.0|679350.0| 790000|         -|
| 315000|647333.3333333334|742850.0| 825000|         -|
| 292500|         564250.0|786600.0| 315000|         -|
| 329000|         509900.0|839200.0| 292500|         -|
| 765000|         479750.0|868450.0| 329000|         -|
| 927000|         520500.0|805750.0| 765000|         -|
|1380000|         571312.5|715250.0| 927000|         -|
| 740000|661166.6666666666|756250.0|1380000|         -|
| 720000|         669050.0|741750.0| 740000|         -|
| 690000|         708350.0|730550.0| 720000|         -|
| 635000|         698350.0|755050.0| 690000|         -|
| 950000|         679350.0|701050.0| 635000|         -|
| 730000|         742850.0|729550.0| 950000|    

# Задание 4

In [19]:
# Создаём датасет для примеров
dataset_1 = [
  {
    'key_1' : 'abc',
    'value_1' : 10,
    'value_2' : 20
  },
  {
    'key_1' : 'def',
    'value_1' : 100,
    'value_2' : 300
  }
]

dataset_2 = [
  {
    'key_2' : 'abc',
    'value_1' : 5.5,
    'value_2' : 2.2
  },
  {
    'key_2' : 'xyz',
    'value_1' : 10.1,
    'value_2' : 13.5
  }
]

df1 = spark.createDataFrame(dataset_1)
print('df1')
df1.show()

df2 = spark.createDataFrame(dataset_2)
print('df2')
df2.show()

df1
+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  abc|     10|     20|
|  def|    100|    300|
+-----+-------+-------+

df2
+-----+-------+-------+
|key_2|value_1|value_2|
+-----+-------+-------+
|  abc|    5.5|    2.2|
|  xyz|   10.1|   13.5|
+-----+-------+-------+



## Задание 4.1

In [ ]:
# Создайте джойн, чтобы получить следующую таблицу
# +---+-------+-------+
# |key|value_1|value_2|
# +---+-------+-------+
# |abc|     10|     20|
# +---+-------+-------+

In [20]:
df1.join(df2, on=df1.key_1 == df2.key_2, how='leftsemi').show()

+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  abc|     10|     20|
+-----+-------+-------+



## Задание 4.2

In [ ]:
# Создайте джойн, чтобы получить следующую таблицу
# +---+-------+-------+
# |key|value_1|value_2|
# +---+-------+-------+
# |def|    100|    300|
# +---+-------+-------+

In [21]:
df1.join(df2, on=df1.key_1 == df2.key_2, how='leftanti').show()

+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  def|    100|    300|
+-----+-------+-------+



## Задание 4.3

In [ ]:
# Создайте Inner джойн с условиями ---hidden---, для df1 и df2, соответсвенно
# В  итоге получится таблица
# +---+-------+-------+---+-------+-------+
# |key|value_1|value_2|key|value_1|value_2|
# +---+-------+-------+---+-------+-------+
# |abc|     10|     20|abc|    5.5|    2.2|
# |abc|     10|     20|xyz|   10.1|   13.5|
# |def|    100|    300|abc|    5.5|    2.2|
# |def|    100|    300|xyz|   10.1|   13.5|
# +---+-------+-------+---+-------+-------+

In [22]:
df1.join(df2, on=df1.value_2 > df2.value_2, how='inner').show()

+-----+-------+-------+-----+-------+-------+
|key_1|value_1|value_2|key_2|value_1|value_2|
+-----+-------+-------+-----+-------+-------+
|  abc|     10|     20|  abc|    5.5|    2.2|
|  abc|     10|     20|  xyz|   10.1|   13.5|
|  def|    100|    300|  abc|    5.5|    2.2|
|  def|    100|    300|  xyz|   10.1|   13.5|
+-----+-------+-------+-----+-------+-------+



In [23]:
spark.stop()